In [16]:
from absbox import API
#localAPI = API("https://absbox.org/api/latest")
localAPI = API("http://localhost:8081/")

In [27]:
from absbox.local.generic import Generic

test01 = Generic(
    "liquidation provider with interest"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":1600
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),("acc02",{"balance":0}))
    ,(("A1",{"balance":1000
             ,"rate":0.08
             ,"originBalance":1000
             ,"originRate":0.08
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.09}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":500
             ,"rate":0.05
             ,"originBalance":500
             ,"originRate":0.05
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.10}
             ,"bondType":{"Sequential":None}
             }))
    ,tuple()
    ,{"amortizing":[
         ["calcInt","A1"]
         ,["liqAccrue","insuranceProvider"]
         ,["liqSupport", "insuranceProvider","acc01"
           ,("Max"
             ,("substract",("bondDueInt","A1","B"),("accountBalance","acc01"))
             ,("constant",0.0)
            )]
         ,["payInt","acc01",["A1","B"]]
         ,["payPrin","acc02",["A1"]]
         ,["payPrin","acc02",["B"]]
         ,[[("bondBalance","A1","B"),"=",0]
           ,["payInt","acc02",["A1","B"]]
           ,["liqRepay","acc01","insuranceProvider"]
           ,["liqRepay","acc02","insuranceProvider"]]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc02"]
      ,["CollectedPrepayment","acc02"]
      ,["CollectedRecoveries","acc02"]]
    ,None
    ,{"insuranceProvider":
         {"lineOfCredit":100,"start":"2021-06-15","fixRate":0.01
          ,"rateAccDates":"MonthEnd","lastAccDate":"2021-06-15"}
     })


In [39]:
from absbox.local.generic import Generic

test01 = Generic(
    "liquidation provider with interest"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":1200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),("acc02",{"balance":0}))
    ,(("A1",{"balance":1000
             ,"rate":0.08
             ,"originBalance":1000
             ,"originRate":0.08
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.09}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":500
             ,"rate":0.05
             ,"originBalance":500
             ,"originRate":0.05
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.10}
             ,"bondType":{"Sequential":None}
             }))
    ,tuple()
    ,{"amortizing":[
         ["calcInt","A1"]
         ,["liqSupport", "insuranceProvider","acc01"
           ,("Max"
             ,("substract",("bondDueInt","A1","B"),("accountBalance","acc01"))
             ,("constant",0.0)
            )]
         ,["payInt","acc01",["A1","B"]]
         ,["payPrin","acc02",["A1"]]
         ,["payPrin","acc02",["B"]]
         ,[[("bondBalance","A1","B"),"=",0]
           ,["payInt","acc02",["A1","B"]]
           ,["liqRepay","acc01","insuranceProvider"]
           ,["liqRepay","acc02","insuranceProvider"]]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc02"]
      ,["CollectedPrepayment","acc02"]
      ,["CollectedRecoveries","acc02"]]
    ,None
    ,{"insuranceProvider":
         {"lineOfCredit":100,"start":"2021-06-15"
          ,"type":{"Reset":"MonthEnd","Formula":("bondBalance",),"Pct":0.0015}}
     })


In [40]:
r = localAPI.run(test01,read=True
                 ,assumptions=[{"CDR":0.0}
                              ,{"Inspect":[(["DayOfMonth",20],("bondDueInt","A1","B"))
                                          ,(["DayOfMonth",20],("accountBalance","acc01"))
                                          ]}
                              ])
r['liqProvider']['insuranceProvider']

,balance,change,used,memo
date,,,,
2021-07-26,2.25,0.00,0.00,
2021-08-20,1.98,0.00,0.00,
2021-09-20,0.00,1.90,1.90,
2021-10-20,0.00,1.81,3.71,
2021-11-20,0.00,1.72,5.43,
2021-12-20,0.00,1.63,7.06,
2022-01-20,0.00,1.54,8.60,
2022-02-20,0.00,1.45,10.05,
2022-03-20,0.00,1.36,11.41,


In [10]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-07-26,767.59,8.98,232.41,0.08,241.39,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-08-20,689.53,4.20,78.06,0.08,82.26,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-09-20,611.18,4.68,78.35,0.08,83.03,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-10-20,532.54,4.01,78.64,0.08,82.65,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-11-20,453.60,3.34,78.94,0.08,82.28,"[<PayInt:[""A1""],Just 0.27>, <PayPrin:[""A1""],Ju..."
2021-12-20,374.37,2.35,79.23,0.08,81.58,"[<PayInt:[""A1""],Just 0.90>, <PayPrin:[""A1""],Ju..."
2022-01-20,294.83,2.25,79.54,0.08,81.79,"[<PayInt:[""A1""],Just 1.19>, <PayPrin:[""A1""],Ju..."
2022-02-20,215.00,2.11,79.83,0.08,81.94,"[<PayInt:[""A1""],Just 1.09>, <PayPrin:[""A1""],Ju..."
2022-03-20,134.86,1.54,80.14,0.08,81.68,"[<PayInt:[""A1""],Just 0.87>, <PayPrin:[""A1""],Ju..."


In [171]:
r['bonds']['B']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-07-26,500.00,2.80,0.00,0.05,2.80,"[<PayInt:[""B""],Just 0.00>, <PayPrin:[""B""],Just..."
2021-08-20,500.00,1.71,0.00,0.05,1.71,"[<PayInt:[""B""],Just 0.00>, <PayPrin:[""B""],Just..."
2021-09-20,500.00,2.12,0.00,0.05,2.12,"[<PayInt:[""B""],Just 0.00>, <PayPrin:[""B""],Just..."
2021-10-20,500.00,2.05,0.00,0.05,2.05,"[<PayInt:[""B""],Just 0.00>, <PayPrin:[""B""],Just..."
2021-11-20,500.00,1.96,0.00,0.05,1.96,"[<PayInt:[""B""],Just 0.16>, <PayPrin:[""B""],Just..."
2021-12-20,500.00,1.59,0.00,0.05,1.59,"[<PayInt:[""B""],Just 0.62>, <PayPrin:[""B""],Just..."
2022-01-20,500.00,1.80,0.00,0.05,1.80,"[<PayInt:[""B""],Just 0.94>, <PayPrin:[""B""],Just..."
2022-02-20,500.00,2.02,0.00,0.05,2.02,"[<PayInt:[""B""],Just 1.04>, <PayPrin:[""B""],Just..."
2022-03-20,500.00,1.90,0.00,0.05,1.90,"[<PayInt:[""B""],Just 1.06>, <PayPrin:[""B""],Just..."


In [11]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [12]:
r['accounts']['acc01']

,balance,change,memo
date,,,
2021-06-15,17.13,17.13,<PoolInflow:CollectedInterest>
2021-07-26,17.13,0.00,<Support:insuranceProvider>
2021-07-26,5.35,-11.78,"<PayInt:[""A1"",""B""],Nothing>"
2021-07-31,10.47,5.12,<PoolInflow:CollectedInterest>
2021-08-20,10.47,0.00,<Support:insuranceProvider>
2021-08-20,4.56,-5.91,"<PayInt:[""A1"",""B""],Nothing>"
2021-08-31,9.39,4.83,<PoolInflow:CollectedInterest>
2021-09-20,9.39,0.00,<Support:insuranceProvider>
2021-09-20,2.59,-6.80,"<PayInt:[""A1"",""B""],Nothing>"


In [13]:
r['accounts']['acc02']

,balance,change,memo
date,,,
2021-06-15,232.41,232.41,<PoolInflow:CollectedPrincipal>
2021-06-15,232.41,0.00,<PoolInflow:CollectedPrepayment>
2021-06-15,232.41,0.00,<PoolInflow:CollectedRecoveries>
2021-07-26,0.00,-232.41,"<PayPrin:[""A1""],Nothing>"
2021-07-26,0.00,0.00,"<PayPrin:[""B""],Nothing>"
2021-07-31,78.06,78.06,<PoolInflow:CollectedPrincipal>
2021-07-31,78.06,0.00,<PoolInflow:CollectedPrepayment>
2021-07-31,78.06,0.00,<PoolInflow:CollectedRecoveries>
2021-08-20,0.00,-78.06,"<PayPrin:[""A1""],Nothing>"


In [166]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum
Date,,,,,,,,,
2021-06-15,1282.11,217.89,16.05,0,0,0,0,0.045,None
2021-07-31,1208.93,73.18,4.80,0,0,0,0,0.045,None
2021-08-31,1135.48,73.45,4.53,0,0,0,0,0.045,None
2021-09-30,1061.75,73.73,4.25,0,0,0,0,0.045,None
2021-10-31,987.75,74.00,3.98,0,0,0,0,0.045,None
2021-11-30,913.46,74.29,3.70,0,0,0,0,0.045,None
2021-12-31,838.90,74.56,3.42,0,0,0,0,0.045,None
2022-01-31,764.05,74.85,3.14,0,0,0,0,0.045,None
2022-02-28,688.93,75.12,2.86,0,0,0,0,0.045,None
